<a href="https://colab.research.google.com/github/gaeunkim0721/cardrecognitionproject/blob/main/_ptToonnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ONNX

In [ ]:
# clone YOLOv5 and reset to a specific git checkpoint that has been verified working
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard 68211f72c99915a15855f7b99bf5d93f5631330f

Cloning into 'yolov5'...
remote: Enumerating objects: 5784, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 5784 (delta 2), reused 2 (delta 0), pack-reused 5770
Receiving objects: 100% (5784/5784), 8.26 MiB | 6.08 MiB/s, done.
Resolving deltas: 100% (3946/3946), done.
/content/yolov5
HEAD is now at 68211f7 FROM nvcr.io/nvidia/pytorch:20.10-py3 (#1553)


In [ ]:
!cp /content/drive/MyDrive/20210420YOLOV5ID/best.pt .

In [ ]:
%matplotlib inline

In [ ]:
# Some standard imports
import io
import numpy as np

from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx

In [ ]:
import torch.nn as nn
import torch.nn.init as init


class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor, inplace=False):
        super(SuperResolutionNet, self).__init__()

        self.relu = nn.ReLU(inplace=inplace)
        self.conv1 = nn.Conv2d(1, 64, (5, 5), (1, 1), (2, 2))
        self.conv2 = nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
        self.conv4 = nn.Conv2d(32, upscale_factor ** 2, (3, 3), (1, 1), (1, 1))
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

        self._initialize_weights()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pixel_shuffle(self.conv4(x))
        return x

    def _initialize_weights(self):
        init.orthogonal_(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv4.weight)

# Create the super-resolution model by using the above model definition.
torch_model = SuperResolutionNet(upscale_factor=3)

In [ ]:
# Load pretrained model weights
model_url = '/content/drive/MyDrive/20210420YOLOV5ID/best.pt'
batch_size = 1    # just a random number

# Initialize model with the pretrained weights
map_location = lambda storage, loc: storage
if torch.cuda.is_available():
    map_location = None
torch_model.load_state_dict(model_zoo.load_url(model_url, map_location=map_location))

# set the model to inference mode
torch_model.eval()

NameError: ignored

In [ ]:
pip uninstall utils

Uninstalling utils-1.0.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/tests/*
    /usr/local/lib/python3.7/dist-packages/utils-1.0.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/utils/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/tests/download_media.py
    /usr/local/lib/python3.7/dist-packages/tests/test_AudioClips.py
    /usr/local/lib/python3.7/dist-packages/tests/test_ImageSequenceClip.py
    /usr/local/lib/python3.7/dist-packages/tests/test_PR.py
    /usr/local/lib/python3.7/dist-packages/tests/test_TextClip.py
    /usr/local/lib/python3.7/dist-packages/tests/test_VideoClip.py
    /usr/local/lib/python3.7/dist-packages/tests/test_VideoFileClip.py
    /usr/local/lib/python3.7/dist-packages/tests/test_Videos.py
    /usr/local/lib/python3.7/dist-packages/tests/test_classifier/__init__.py
    /usr/local/lib/python3.7/dist-packages/tests/test_classifier/conftest.py
    /usr/local/lib/python3.7/dist-packages/tests/te

In [ ]:
pip install onnx

     |████████████████████████████████| 12.2MB 4.2MB/s 


In [ ]:
!python /content/yolov5/models/export.py

Namespace(batch_size=1, img_size=[640, 640], weights='/content/yolov5/best.pt')
Fusing layers... 
Model Summary: 316 layers, 21492876 parameters, 0 gradients

Starting TorchScript export with torch 1.8.1+cu101...
/usr/local/lib/python3.7/dist-packages/torch/jit/_trace.py:940: TracerWarning: Encountering a list at the output of the tracer might cause the trace to be incorrect, this is only valid if the container structure does not change based on the module's inputs. Consider using a constant container instead (e.g. for `list`, use a `tuple` instead. for `dict`, use a `NamedTuple` instead). If you absolutely need this and know the side effects, pass strict=False to trace() to allow this behavior.
  _force_outplace,
TorchScript export success, saved as /content/yolov5/best.torchscript.pt

Starting ONNX export with onnx 1.9.0...
ONNX export success, saved as /content/yolov5/best.onnx
CoreML export failure: No module named 'coremltools'

Export complete (17.13s). Visualize with https://git

In [ ]:
!cp /content/yolov5/best.onnx /content/drive/MyDrive/20210420YOLOV5ID/